# Elements logiciels pour le traitement des données en grande dimension

## Installation de pyspark

## Arbre k-d classique

### Création d'un jeu de données

In [10]:
import numpy as np
import pandas

n = 10
data = pandas.DataFrame(np.random.rand(n,2)*100,  columns = ['X', 'Y'])
data

,X,Y
0,79.095012,13.124849
1,26.563985,5.025022
2,32.391453,92.698101
3,89.067685,36.580554
4,64.863779,31.188360
5,67.515052,94.241552
6,0.494278,82.969893
7,38.321266,23.547071
8,96.241266,8.752980
9,8.494980,86.954100


In [14]:
data.index.values.tolist()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]